This file first shows how to run an inelastic Static Traffic Assignment in dyntapy.

It also shows examples of how to run an inelastic assignment with tolls.

1. How to run an inelastic traffic assignment

In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import numpy as np
import os
import sys
sys.path.append("../../..")

from pickle import dump
from dyntapy.supply_data import get_toy_network, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from dyntapy.visualization import show_network, show_demand
from dyntapy.toll import create_toll_object

First, we retrieve a toy network (cascetta).

Next, we create centroids, which are added to the network.

Lastly, we create a made-up OD-matrix, which is loaded onto the centroids. 

In [ ]:
# 1. Retrieve network 
g = get_toy_network('cascetta')

# 2. Create and add centroids
centroid_x = np.array([2+np.sqrt(2), 5, np.sqrt(2)])
centroid_y = np.array([-0.5, np.sqrt(2), np.sqrt(2)])
g = add_centroids(g, centroid_x, centroid_y, euclidean=True)
# also adds connectors automatically
g = relabel_graph(g)  # adding link and node ids, connectors and centroids
show_network(g, euclidean=True, notebook=True)

# 3. Create OD and load onto centroids 
od_matrix = np.zeros(9).reshape((3, 3))
od_matrix[0, 1] = 100
od_matrix[2, 1] = 100
od_graph = od_graph_from_matrix(od_matrix, centroid_x, centroid_y)
show_demand(od_graph, euclidean=True, notebook=True)

Next code block shows how to run an STA without tolls. 

In [ ]:
assignment = StaticAssignment(g, od_graph)
result = assignment.run('dial_b')
print('dial_b ran successfully')
show_network(g, flows = result.flows, notebook=True, show_nodes=False, euclidean=True)

The next code block shows an example of how to run STAs with tolls. Different toll schemes can be used, for which we refer to the tolling file. Look at the output of the previous cell block and this cell block. 

In [ ]:
# Create toll object
toll_method = 'zone'
toll_link_ids = [9,10] # Link 9 goes from the bottom to the right, and link 10 goes from the bottom to the top. 
toll_value = 3
toll_object = create_toll_object(g, toll_method, toll_link_ids, toll_value)

# Run STA with toll object
assignment2 = StaticAssignment(g,od_graph, toll_object)
result2 = assignment2.run('dial_b')
print('dial_b ran successfully')
show_network(g, flows = result2.flows, notebook=True, show_nodes=False, euclidean=True)


-----------------------------------------------------------------------------------------------------------------------


2. HEEDS preparation

As HEEDS uses an STA in every iteration, we would like to save the network with the centroids and the od_graph. Otherwise, these should be computed in every iteration, while they are static throughout the whole process. 

In [ ]:
data_path = os.path.dirname(os.path.realpath("__file__"))+ os.path.sep+os.pardir+os.path.sep+os.pardir+os.path.sep+'data'
HEEDS_data_path = data_path + os.path.sep + "HEEDS_input"

network_path = HEEDS_data_path + os.path.sep + 'network_centroids_data' + os.path.sep + "inelastic_toy"
graph_path = HEEDS_data_path + os.path.sep + "od_graph_data" + os.path.sep + "inelastic_toy"

with open(network_path, 'wb') as network_file:
    dump(g, network_file)
    print(f'network saved at f{network_path}')
with open(graph_path, 'wb') as f:
    dump(od_graph,f)
    print(f'od_graph saved at f{graph_path}')